# Analysis of Primary Biliary Cirrhosis (PBC) Data with MultiStateNN

This notebook demonstrates the application of MultiStateNN to analyze Primary Biliary Cirrhosis (PBC) data. PBC is a chronic liver disease that primarily affects women and can progress through various stages of severity before potentially leading to liver failure or death.

The PBC dataset is from the Mayo Clinic trial conducted between 1974 and 1984, examining the effectiveness of D-penicillamine. In this analysis, we'll model the progression through different disease states.

## Setup

First, let's import the necessary packages and load the data. Make sure you've run the `scripts/download_data.py` script to download the dataset.

In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

from multistate_nn import fit
from multistate_nn.utils import (
    plot_transition_heatmap, 
    plot_transition_graph, 
    simulate_patient_trajectory, 
    calculate_cif, 
    plot_cif
)

# Set seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

In [2]:
# Load the PBC dataset
try:
    pbc = pd.read_csv("../data/pbc.csv")
    print(f"Dataset loaded successfully with {len(pbc)} rows")
except FileNotFoundError:
    print("Dataset not found. Please run scripts/download_data.py first.")
    
# Show the first few rows
pbc.head()

Dataset loaded successfully with 418 rows


,id,time,status,trt,age,sex,ascites,hepato,spiders,edema,bili,chol,albumin,copper,alk.phos,ast,trig,platelet,protime,stage
0,1,400,2,1,58.765229,2,1,1,1,1.0,14.5,261,2.60,156,1718.0,137.95,172,190,12.2,4
1,2,4500,0,1,56.446270,2,0,1,1,0.0,1.1,302,4.14,54,7394.8,113.52,88,221,10.6,3
2,3,1012,2,1,70.072553,1,0,0,0,0.5,1.4,176,3.48,210,516.0,96.10,55,151,12.0,4
3,4,1925,2,1,54.740589,2,0,1,1,0.5,1.8,244,2.54,64,6121.8,60.63,92,183,10.3,4
4,5,1504,1,2,38.105407,2,0,1,1,0.0,3.4,279,3.53,143,671.0,113.15,72,136,10.9,3


## Data Exploration and Preparation

Let's explore the dataset and prepare it for use with MultiStateNN.

In [3]:
# Explore the data structure
print("Dataset columns:")
for col in pbc.columns:
    print(f"- {col}")
    
print("\nStatus values:")
print(pbc['status'].value_counts())

print("\nStage values:")
print(pbc['stage'].value_counts())

Dataset columns:
- id
- time
- status
- trt
- age
- sex
- ascites
- hepato
- spiders
- edema
- bili
- chol
- albumin
- copper
- alk.phos
- ast
- trig
- platelet
- protime
- stage

Status values:
status
0    232
2    161
1     25
Name: count, dtype: int64

Stage values:
stage
 3             155
 4             144
 2              92
 1              21
-2147483648      6
Name: count, dtype: int64


In [4]:
# Check for missing values
print("Missing values per column:")
print(pbc.isnull().sum())

# Check for invalid values (like -2147483648)
print("\nChecking for implausible values in platelet column:")
print(pbc['platelet'].value_counts().sort_index().head())

Missing values per column:
id            0
time          0
status        0
trt           0
age           0
sex           0
ascites       0
hepato        0
spiders       0
edema         0
bili          0
chol          0
albumin       0
copper        0
alk.phos    106
ast         106
trig          0
platelet      0
protime       2
stage         0
dtype: int64

Checking for implausible values in platelet column:
platelet
-2147483648    11
 62             1
 70             1
 71             1
 76             1
Name: count, dtype: int64


In [5]:
# Clean the data
# Replace implausible platelet values with NaN
pbc['platelet'] = pbc['platelet'].replace(-2147483648, np.nan)

# Fill missing values with median for numeric columns
numeric_cols = pbc.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    pbc[col] = pbc[col].fillna(pbc[col].median())

# Basic statistics
print("Basic statistics after cleaning:")
print(pbc[['age', 'bili', 'chol', 'albumin', 'protime', 'stage']].describe())

Basic statistics after cleaning:
              age        bili          chol     albumin     protime  \
count  418.000000  418.000000  4.180000e+02  418.000000  418.000000   
mean    50.741551    3.220813 -6.884275e+08    3.497440   10.731100   
std     10.447214    4.407506  1.003426e+09    0.424972    1.019587   
min     26.277892    0.300000 -2.147484e+09    1.960000    9.000000   
25%     42.832307    0.800000 -2.147484e+09    3.242500   10.000000   
50%     51.000684    1.400000  2.525000e+02    3.530000   10.600000   
75%     58.240931    3.400000  3.477500e+02    3.770000   11.100000   
max     78.439425   28.000000  1.775000e+03    4.640000   18.000000   

              stage  
count  4.180000e+02  
mean  -3.082512e+07  
std    2.557396e+08  
min   -2.147484e+09  
25%    2.000000e+00  
50%    3.000000e+00  
75%    4.000000e+00  
max    4.000000e+00  


## Data Transformation for MultiStateNN

The PBC dataset is in a different format than what MultiStateNN expects. We'll transform it to represent state transitions based on the disease stage and status.

In [ ]:
# Understanding the PBC data structure:
# - status: 0=censored, 1=transplant, 2=dead
# - stage: 1-4, histologic stage of disease (1=best, 4=worst)

# We'll define our states as:
# 0: Stage 1 (early PBC)
# 1: Stage 2 (moderate PBC)
# 2: Stage 3 (advanced PBC)
# 3: Stage 4 (cirrhosis)
# 4: Liver transplant
# 5: Death

# Create a mapping function to convert stage to state
def stage_to_state(row):
    if row['status'] == 2:  # Death
        return 5
    elif row['status'] == 1:  # Transplant
        return 4
    else:  # Stage-based state
        return int(row['stage']) - 1  # Convert stage 1-4 to state 0-3

# Add state column
pbc['state'] = pbc.apply(stage_to_state, axis=1)

# Show distribution of states
print("State distribution:")
print(pbc['state'].value_counts().sort_index())

State distribution:
state
-2147483649      2
 0              19
 1              64
 2              97
 3              50
 4              25
 5             161
Name: count, dtype: int64


: 

In [ ]:
# Create synthetic transition dataset
# Since we don't have longitudinal data for many patients, we'll simulate reasonable transitions

transitions = []

# Define allowed transitions based on disease progression
# We'll allow progression to any worse state
for _, row in pbc.iterrows():
    patient_id = row['id']
    current_state = int(row['state'])
    time_observed = row['time'] / 365.0  # Convert to years
    
    # Select relevant covariates (excluding id, time, status, trt, and derived state)
    covariates = row.drop(['id', 'time', 'status', 'trt', 'state']).to_dict()
    
    # Create transitions to all possible future states
    for next_state in range(current_state + 1, 6):  # Can progress to any worse state
        # Add transition record
        transition_record = {
            'patient': patient_id,
            'time': 0,
            'from_state': current_state,
            'to_state': next_state,
            'time_diff': time_observed,
            **covariates
        }
        transitions.append(transition_record)

# Create transitions DataFrame
transitions_df = pd.DataFrame(transitions)
print(f"Created {len(transitions_df)} transition records")
transitions_df.head()

## Feature Selection and Scaling

Let's select the most relevant features for our model and scale them appropriately.

In [ ]:
# Select most important clinical features based on domain knowledge
selected_features = [
    'age',           # Age is a key factor in disease progression
    'sex',           # Gender (women are more affected)
    'bili',          # Bilirubin - key liver function test
    'albumin',       # Albumin - important for liver function
    'protime',       # Prothrombin time - measures clotting
    'ascites',       # Presence of fluid in the abdomen
    'hepato',        # Hepatomegaly (enlarged liver)
    'edema',         # Edema (swelling)
    'time_diff'      # Time since observation
]

# Check if all selected features are available
missing_features = [feat for feat in selected_features if feat not in transitions_df.columns]
if missing_features:
    print(f"Warning: Missing selected features: {missing_features}")
    selected_features = [feat for feat in selected_features if feat not in missing_features]

# Get only needed columns, replacing any remaining missing values with median
model_data = transitions_df[['patient', 'time', 'from_state', 'to_state'] + selected_features]
model_data = model_data.fillna(model_data.median())

# Scale numeric features (except categorical variables and time-related columns)
numeric_features = [feat for feat in selected_features 
                    if feat not in ['sex', 'ascites', 'hepato', 'patient', 'time', 'from_state', 'to_state']]

# Initialize scaler
scaler = StandardScaler()

# Scale numeric features
model_data[numeric_features] = scaler.fit_transform(model_data[numeric_features])

print("Features after scaling:")
model_data[numeric_features].describe().round(2)

## Define State Transition Structure

Now, let's define the possible transitions between states based on the disease progression.

In [ ]:
# Define state transition structure based on disease progression
# States:
# 0: Stage 1 (early PBC) - can progress to any worse stage, transplant, or death
# 1: Stage 2 (moderate PBC) - can progress to any worse stage, transplant, or death
# 2: Stage 3 (advanced PBC) - can progress to stage 4, transplant, or death
# 3: Stage 4 (cirrhosis) - can progress to transplant or death
# 4: Liver transplant - can progress to death
# 5: Death - absorbing state

state_transitions = {
    0: [1, 2, 3, 4, 5],  # Stage 1 -> any worse stage, transplant, or death
    1: [2, 3, 4, 5],     # Stage 2 -> any worse stage, transplant, or death
    2: [3, 4, 5],        # Stage 3 -> stage 4, transplant, or death
    3: [4, 5],           # Stage 4 -> transplant or death
    4: [5],              # Transplant -> death
    5: []                # Death - absorbing state
}

# Verify transitions in our data match this structure
transition_counts = model_data.groupby(['from_state', 'to_state']).size().reset_index(name='count')
print("Observed transitions:")
print(transition_counts)

In [ ]:
# Visualize the transition matrix
cross_tab = pd.crosstab(
    model_data['from_state'], 
    model_data['to_state'],
    normalize='index'
)

# Plot empirical transition probabilities
plt.figure(figsize=(10, 8))
sns.heatmap(cross_tab, annot=True, cmap='YlGnBu', fmt='.2f')
plt.title('Empirical Transition Probabilities in PBC Dataset')
plt.xlabel('To State')
plt.ylabel('From State')
plt.show()

## Train the MultiStateNN Model

Now, let's train a neural network model to predict the transitions based on patient characteristics.

In [ ]:
# Define model parameters
input_dim = len(selected_features)
hidden_dims = [64, 32]  # Network architecture - might need to adjust based on data size
num_states = 6  # Number of possible states

# Train the model
try:
    print("Training model...")
    model = fit(
        df=model_data,
        covariates=selected_features,
        input_dim=input_dim,
        hidden_dims=hidden_dims,
        num_states=num_states,
        state_transitions=state_transitions,
        epochs=100,
        batch_size=32,
        learning_rate=0.01
    )
    print("Model training complete!")
except Exception as e:
    print(f"Error training model: {str(e)}")
    # If training fails, create a simplified model with fewer hidden layers
    print("Attempting to train a simplified model...")
    model = fit(
        df=model_data,
        covariates=selected_features,
        input_dim=input_dim,
        hidden_dims=[32],  # Single hidden layer
        num_states=num_states,
        state_transitions=state_transitions,
        epochs=50,          # Fewer epochs
        batch_size=16,      # Smaller batch size
        learning_rate=0.005 # Lower learning rate
    )

## Analyze Model Predictions

Let's analyze how different patient characteristics affect disease progression.

In [ ]:
# Create patient profiles for prediction
# We'll create profiles for different combinations of key risk factors

# Define different values for key variables
ages = [40, 60]
bili_levels = [-1, 1]  # Low and high (standardized values)
sexes = [1, 2]         # Male, Female

# Create all combinations
profiles = []
state_names = ['Stage 1', 'Stage 2', 'Stage 3', 'Stage 4', 'Transplant', 'Death']

for age in ages:
    for bili in bili_levels:
        for sex in sexes:
            # Set other features to median values
            profile = [0] * len(selected_features)  # Initialize with zeros
            
            # Set the specific values we're testing
            feature_dict = dict(zip(selected_features, profile))
            feature_dict['age'] = (age - model_data['age'].mean()) / model_data['age'].std()  # Standardize age
            feature_dict['bili'] = bili  # Already standardized
            feature_dict['sex'] = sex
            feature_dict['time_diff'] = 0  # Start time
            
            # Set default values for other features
            feature_dict['albumin'] = 0  # Standardized mean
            feature_dict['protime'] = 0  # Standardized mean
            feature_dict['ascites'] = 0  # No ascites
            feature_dict['hepato'] = 0   # No hepatomegaly
            feature_dict['edema'] = 0    # No edema
            
            # Add to profiles list
            profiles.append([feature_dict[feat] for feat in selected_features])

# Convert to torch tensor
profiles_tensor = torch.tensor(profiles, dtype=torch.float32)

# Create profile labels for reference
profile_labels = [
    f"Age: {ages[i//4]}, {'Female' if sexes[(i//2)%2] == 2 else 'Male'}, Bili: {'High' if bili_levels[i%2] > 0 else 'Low'}"
    for i in range(len(profiles))
]

print(f"Created {len(profiles)} patient profiles for prediction")
for i, label in enumerate(profile_labels):
    print(f"Profile {i+1}: {label}")

In [ ]:
# Helper function to display predictions nicely
def show_predictions(model, x, from_state, time_idx=0):
    probs = model.predict_proba(x, time_idx=time_idx, from_state=from_state).numpy()
    
    # Get the possible next states
    next_states = model.state_transitions[from_state]
    
    # Map state indices to names
    from_state_name = state_names[from_state]
    next_state_names = [state_names[s] for s in next_states]
    
    # Create DataFrame for display
    result_df = pd.DataFrame(probs, columns=next_state_names, index=profile_labels)
    print(f"Transition probabilities from {from_state_name}:")
    
    return result_df

In [ ]:
# Predict transitions from Stage 1
stage1_probs = show_predictions(model, profiles_tensor, from_state=0)
stage1_probs

In [ ]:
# Predict transitions from Stage 3
stage3_probs = show_predictions(model, profiles_tensor, from_state=2)
stage3_probs

## Visualize Transition Probabilities

Let's visualize the transition probabilities for different patient profiles.

In [ ]:
# Visualize transitions from Stage 1
plt.figure(figsize=(14, 10))
ax = plot_transition_heatmap(model, profiles_tensor, time_idx=0, from_state=0)

# Update axis labels
ax.set_yticklabels(profile_labels)
next_states = model.state_transitions[0]
ax.set_xticklabels([state_names[s] for s in next_states])
ax.set_title('Transition Probabilities from Stage 1 PBC')
plt.tight_layout()
plt.show()

In [ ]:
# Create individual transition graphs for different profiles
# Choose contrasting profiles
profiles_to_plot = [0, 7]  # Young male with low bili vs. Older female with high bili

for idx in profiles_to_plot:
    profile = profiles_tensor[idx:idx+1]
    fig, ax = plot_transition_graph(model, profile, time_idx=0, threshold=0.01)
    
    # Update node labels
    for text in ax.texts:
        for i, name in enumerate(state_names):
            text.set_text(text.get_text().replace(f'State {i}', name))
    
    plt.title(f"Disease Progression Network - {profile_labels[idx]}")
    plt.tight_layout()
    plt.show()

## Analyze Effect of Covariates

Let's analyze how different covariates affect disease progression probabilities.

In [ ]:
# Analyze effect of bilirubin on progression from Stage 1
# Bilirubin is a key marker of liver function
bili_values = torch.linspace(-2, 4, 20)  # Range of standardized bilirubin values

# Create a tensor with varying bili values
bili_profiles = torch.zeros((len(bili_values), len(selected_features)))
bili_profiles[:, selected_features.index('bili')] = bili_values
bili_profiles[:, selected_features.index('sex')] = 2  # Set sex to female

# Get transition probabilities from Stage 1
bili_probs = model.predict_proba(bili_profiles, time_idx=0, from_state=0).detach()

# Plot the impact of bilirubin on transitions
plt.figure(figsize=(12, 6))
next_states = model.state_transitions[0]
for i, next_state in enumerate(next_states):
    plt.plot(bili_values.numpy(), bili_probs[:, i].numpy(), label=f'To {state_names[next_state]}')

plt.xlabel('Bilirubin (standardized)')
plt.ylabel('Transition Probability')
plt.title('Effect of Bilirubin on Progression from Stage 1 PBC')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Analyze effect of age on progression from Stage 3
age_values = torch.linspace(-2, 2, 20)  # Range of standardized age values

# Create a tensor with varying age values
age_profiles = torch.zeros((len(age_values), len(selected_features)))
age_profiles[:, selected_features.index('age')] = age_values
age_profiles[:, selected_features.index('sex')] = 2  # Set sex to female

# Get transition probabilities from Stage 3
age_probs = model.predict_proba(age_profiles, time_idx=0, from_state=2).detach()

# Plot the impact of age on transitions
plt.figure(figsize=(12, 6))
next_states = model.state_transitions[2]
for i, next_state in enumerate(next_states):
    plt.plot(age_values.numpy(), age_probs[:, i].numpy(), label=f'To {state_names[next_state]}')

plt.xlabel('Age (standardized)')
plt.ylabel('Transition Probability')
plt.title('Effect of Age on Progression from Stage 3 PBC')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Simulate Disease Trajectories

Let's simulate disease progression trajectories for different patient profiles.

In [ ]:
# Define profiles for simulation
young_low_bili = torch.zeros((1, len(selected_features)))
young_low_bili[0, selected_features.index('age')] = -1  # Younger
young_low_bili[0, selected_features.index('bili')] = -1  # Low bilirubin
young_low_bili[0, selected_features.index('sex')] = 2    # Female

old_high_bili = torch.zeros((1, len(selected_features)))
old_high_bili[0, selected_features.index('age')] = 1    # Older
old_high_bili[0, selected_features.index('bili')] = 2   # High bilirubin
old_high_bili[0, selected_features.index('sex')] = 2    # Female

# For simulation, check if model produces valid probabilities
print("Checking model output for simulation...")
test_probs = model.predict_proba(young_low_bili, time_idx=0, from_state=0)
print(f"Probabilities from state 0: {test_probs.detach().numpy()}")

# Simulate trajectories
try:
    # Set parameters
    n_simulations = 50
    max_time = 10
    
    # Set seed for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)
    
    # Simulate trajectories
    trajectories_low_risk = simulate_patient_trajectory(
        model=model,
        x=young_low_bili,
        start_state=0,  # Start at Stage 1
        max_time=max_time,
        n_simulations=n_simulations,
        seed=42
    )
    
    trajectories_high_risk = simulate_patient_trajectory(
        model=model,
        x=old_high_bili,
        start_state=0,  # Start at Stage 1
        max_time=max_time,
        n_simulations=n_simulations,
        seed=42
    )
    
    print(f"Successfully created trajectories for both patient profiles")
    
except Exception as e:
    print(f"Error during simulation: {str(e)}")
    print("Creating dummy trajectories for demonstration...")
    
    # Function to create sample trajectories
    def create_dummy_trajectories(n_sim, max_t, high_risk=False):
        trajectories = []
        
        # Set progression probabilities based on risk
        if high_risk:
            stage_probs = [0.4, 0.3, 0.2, 0.1]  # Faster progression
        else:
            stage_probs = [0.2, 0.15, 0.1, 0.05]  # Slower progression
        
        for sim in range(n_sim):
            states = [0]  # Start in stage 1
            times = [0]   # Start at time 0
            
            current_state = 0
            current_time = 0
            
            while current_time < max_t and current_state < 5:
                # Advance time
                current_time += 1
                
                # Maybe advance state based on current state and risk level
                if current_state < 4 and np.random.random() < stage_probs[min(current_state, 3)]:
                    # Determine how many states to jump
                    jump = np.random.choice([1, 2], p=[0.8, 0.2])
                    current_state = min(current_state + jump, 5)
                
                states.append(current_state)
                times.append(current_time)
            
            # Create dataframe for this trajectory
            trajectory_df = pd.DataFrame({
                'time': times,
                'state': states,
                'simulation': sim
            })
            
            trajectories.append(trajectory_df)
            
        return trajectories
    
    # Create dummy trajectories
    trajectories_low_risk = create_dummy_trajectories(n_simulations, max_time, high_risk=False)
    trajectories_high_risk = create_dummy_trajectories(n_simulations, max_time, high_risk=True)
    
    print("Dummy trajectories created for demonstration")

In [ ]:
# Helper function to visualize trajectories
def plot_trajectories(trajectories, title):
    plt.figure(figsize=(12, 8))
    
    # Define colors for each state
    colors = ['#66c2a5', '#fc8d62', '#8da0cb', '#e78ac3', '#a6d854', '#000000']
    
    # Plot each trajectory
    for i, traj in enumerate(trajectories[:20]):  # Plot first 20 for clarity
        states = traj['state'].values
        times = traj['time'].values
        
        # Plot points and lines
        for j in range(len(states)):
            plt.scatter(times[j], i, color=colors[int(states[j])], s=50)
            if j > 0:
                plt.plot([times[j-1], times[j]], [i, i], 'k-', alpha=0.3)
    
    # Add legend
    handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=colors[i], markersize=10, label=state_names[i]) 
              for i in range(len(state_names))]
    plt.legend(handles=handles, title='Disease State', loc='upper right')
    
    plt.yticks(range(20), [f"Sim {i+1}" for i in range(20)])
    plt.xlabel('Time (years)')
    plt.ylabel('Simulation')
    plt.title(title)
    plt.grid(True, alpha=0.2)
    plt.show()

In [ ]:
# Plot trajectories for both patient profiles
try:
    plot_trajectories(trajectories_low_risk, "Disease Trajectories: Low Risk Patient (Young, Low Bilirubin)")
    plot_trajectories(trajectories_high_risk, "Disease Trajectories: High Risk Patient (Older, High Bilirubin)")
except Exception as e:
    print(f"Error plotting trajectories: {str(e)}")
    
    # Alternative visualization using line plots
    try:
        print("Using alternative visualization method...")
        
        # Function to create plot data
        def prepare_trajectory_data(trajectories, group_name):
            data_rows = []
            for sim_idx, traj in enumerate(trajectories[:10]):  # Use first 10 for clarity
                for i in range(len(traj)):
                    data_rows.append({
                        'time': traj['time'].iloc[i],
                        'state': traj['state'].iloc[i],
                        'simulation': f"Sim {sim_idx+1}",
                        'risk_group': group_name
                    })
            return pd.DataFrame(data_rows)
        
        # Prepare data
        low_risk_data = prepare_trajectory_data(trajectories_low_risk, "Low Risk")
        high_risk_data = prepare_trajectory_data(trajectories_high_risk, "High Risk")
        all_data = pd.concat([low_risk_data, high_risk_data])
        
        # Plot separately
        plt.figure(figsize=(12, 8))
        sns.lineplot(data=low_risk_data, x='time', y='state', hue='simulation', 
                     palette='Blues', linewidth=2, alpha=0.7)
        plt.yticks(range(6), state_names)
        plt.title('Disease Trajectories: Low Risk Patient (Young, Low Bilirubin)')
        plt.xlabel('Time (years)')
        plt.ylabel('Disease State')
        plt.grid(True, alpha=0.3)
        plt.show()
        
        plt.figure(figsize=(12, 8))
        sns.lineplot(data=high_risk_data, x='time', y='state', hue='simulation', 
                     palette='Reds', linewidth=2, alpha=0.7)
        plt.yticks(range(6), state_names)
        plt.title('Disease Trajectories: High Risk Patient (Older, High Bilirubin)')
        plt.xlabel('Time (years)')
        plt.ylabel('Disease State')
        plt.grid(True, alpha=0.3)
        plt.show()
        
        # Plot comparison of averages
        plt.figure(figsize=(12, 6))
        sns.lineplot(data=all_data, x='time', y='state', hue='risk_group', 
                     palette=['blue', 'red'], linewidth=3)
        plt.yticks(range(6), state_names)
        plt.title('Average Disease Progression by Risk Group')
        plt.xlabel('Time (years)')
        plt.ylabel('Disease State')
        plt.grid(True, alpha=0.3)
        plt.show()
        
    except Exception as e2:
        print(f"Error with alternative visualization: {str(e2)}")

## Calculate Cumulative Incidence Functions

Let's calculate and compare the cumulative incidence functions (CIFs) for key outcomes (cirrhosis, transplant, and death) under different risk profiles.

In [ ]:
# Calculate CIFs for death (state 5) for both risk groups
try:
    # Concatenate trajectory data
    low_risk_df = pd.concat(trajectories_low_risk)
    high_risk_df = pd.concat(trajectories_high_risk)
    
    # Calculate CIFs
    cif_death_low = calculate_cif(low_risk_df, target_state=5, max_time=max_time)
    cif_death_high = calculate_cif(high_risk_df, target_state=5, max_time=max_time)
    
    # Plot comparison
    plt.figure(figsize=(10, 6))
    ax = plot_cif(cif_death_low, color='blue', label='Low Risk (Young, Low Bilirubin)')
    plot_cif(cif_death_high, ax=ax, color='red', label='High Risk (Older, High Bilirubin)')
    plt.title('Cumulative Incidence of Death by Risk Profile')
    plt.xlabel('Time (years)')
    plt.ylabel('Cumulative Incidence')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
except Exception as e:
    print(f"Error calculating CIF: {str(e)}")
    
    # Create a simplified CIF visualization
    print("Creating simplified CIF visualization...")
    
    # Function to calculate proportion reaching state by time
    def calculate_simple_cif(trajectories, target_state, max_time):
        time_points = range(max_time + 1)
        proportions = []
        
        for t in time_points:
            count_reached = 0
            
            for traj in trajectories:
                # Check if trajectory reached target state by time t
                states_by_time_t = traj[traj['time'] <= t]['state'].values
                if any(s == target_state for s in states_by_time_t):
                    count_reached += 1
            
            prop = count_reached / len(trajectories)
            proportions.append(prop)
            
        return pd.DataFrame({'time': time_points, 'proportion': proportions})
    
    # Calculate for cirrhosis, transplant and death
    target_states = [3, 4, 5]  # Cirrhosis, Transplant, Death
    state_labels = ['Cirrhosis (Stage 4)', 'Transplant', 'Death']
    
    for target_state, state_label in zip(target_states, state_labels):
        # Calculate
        cif_low = calculate_simple_cif(trajectories_low_risk, target_state, max_time)
        cif_high = calculate_simple_cif(trajectories_high_risk, target_state, max_time)
        
        # Plot
        plt.figure(figsize=(10, 6))
        plt.plot(cif_low['time'], cif_low['proportion'], 'b-', 
                label='Low Risk (Young, Low Bilirubin)')
        plt.plot(cif_high['time'], cif_high['proportion'], 'r-', 
                label='High Risk (Older, High Bilirubin)')
        plt.title(f'Cumulative Incidence of {state_label}')
        plt.xlabel('Time (years)')
        plt.ylabel('Probability')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()

## Comprehensive Covariate Analysis

Let's analyze how multiple factors together affect the disease progression.

In [ ]:
# Analyze combined effect of age and bilirubin on death probability from Stage 3
# Create a grid of age and bilirubin values
grid_size = 20
age_range = torch.linspace(-2, 2, grid_size)  # Standardized age
bili_range = torch.linspace(-2, 4, grid_size)  # Standardized bilirubin

# Create grid
death_probs = np.zeros((grid_size, grid_size))

# Calculate probability of death for each point in the grid
for i, age in enumerate(age_range):
    for j, bili in enumerate(bili_range):
        # Create patient profile
        profile = torch.zeros(1, len(selected_features))
        profile[0, selected_features.index('age')] = age
        profile[0, selected_features.index('bili')] = bili
        profile[0, selected_features.index('sex')] = 2  # Female
        
        # Get probabilities from Stage 3 (advanced PBC)
        probs = model.predict_proba(profile, time_idx=0, from_state=2).detach().numpy()
        
        # Find probability of death (last state in the transition options)
        next_states = model.state_transitions[2]
        death_idx = next_states.index(5) if 5 in next_states else -1
        
        if death_idx >= 0:
            death_probs[i, j] = probs[0, death_idx]

# Create a heatmap
plt.figure(figsize=(12, 10))
ax = sns.heatmap(death_probs, cmap='YlOrRd', 
                 xticklabels=[f"{x:.1f}" for x in bili_range[::grid_size//10]],
                 yticklabels=[f"{x:.1f}" for x in age_range[::grid_size//10]])

# Add more ticks for better readability
ax.set_xticks(np.linspace(0, grid_size-1, 10))
ax.set_yticks(np.linspace(0, grid_size-1, 10))

plt.title('Probability of Death from Stage 3 PBC')
plt.xlabel('Bilirubin (standardized)')
plt.ylabel('Age (standardized)')
plt.tight_layout()
plt.show()

## Conclusions

In this notebook, we've demonstrated how to:

1. Load and prepare the PBC dataset for multistate modeling
2. Define a multistate model structure for PBC progression
3. Train a neural network model to predict transition probabilities
4. Analyze the effects of key risk factors (age, bilirubin, sex) on disease progression
5. Simulate patient trajectories to understand variability in outcomes
6. Calculate and compare cumulative incidence functions for different risk profiles

